# Exchanges exploration

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pprint import pprint

import bw2analyzer as bwa
import bw2calc as bc
import bw2data as bd
import bw2io as bi
import bw_processing as bwp
import numpy as np

# import ipywidgets as widgets
import pandas as pd
from bw2data.query import Filter, Query
from IPython.display import display
from tqdm import autonotebook

/tmp/ipykernel_582399/2816604150.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm import autonotebook


In [3]:
from project_details import EI_DB_NAME, PROJECT_NAME

In [4]:
bd.projects.set_current(PROJECT_NAME)
bd.databases

Databases dictionary with 3 object(s):
	asphalt
	biosphere3
	ecoinvent-3.9.1-cutoff

In [5]:
# Is the background database name the same as the one we wrote in `propject_details.py`?
assert EI_DB_NAME in bd.databases

In [6]:
FG_DB_NAME = "asphalt"

In [7]:
db_asphalt = bd.Database(FG_DB_NAME)

In [8]:
pavement_complete_a = db_asphalt.get("DZOAB, A")

In [9]:
bwa.print_recursive_supply_chain(pavement_complete_a, max_level=1)

1: 'DZOAB, A' (kilogram, NL, None)
  1: 'A1, pavement, materials, A' (kilogram, NL, None)
  1: 'A2, pavement, transport to plant, A' (kilogram, NL, None)
  1: 'A3, pavement, production, A' (kilogram, NL, None)
  1: 'A4, pavement, transport to site' (kilogram, NL, None)
  1: 'A5, pavement, construction' (kilogram, NL, None)
  1: 'C1, pavement, demolition' (kilogram, NL, None)
  1: 'C2, pavement, transport to processing' (kilogram, NL, None)
  1: 'C3, pavement, processing' (kilogram, NL, None)


## Associate exchanges to samples

### Work with the pavement materials

First let's verify that we can use the information already contained in the db.

The excel file for the foreground provides a comments field that gives a generic name of the exchange. This can be linked to the name of the column of the generated samples.

Let's first look at the pavement materials data to create the data structures that we could use later with data packages.

In [10]:
pavement_mats = bd.get_activity(25964)
print(pavement_mats)
for e in pavement_mats.technosphere():
    print(e)

'A1, pavement, materials, A' (kilogram, NL, None)
Exchange: 0.3 kilogram 'asphalt granulate, free of burden' (kilogram, NL, None) to 'A1, pavement, materials, A' (kilogram, NL, None)>
Exchange: 0.0412 kilogram 'bitumen adhesive compound production, hot' (kilogram, RER, None) to 'A1, pavement, materials, A' (kilogram, NL, None)>
Exchange: 0.5861 kilogram 'crushed stone, from quarry in Europe, excluding transport to the Netherlands' (kilogram, NL, None) to 'A1, pavement, materials, A' (kilogram, NL, None)>
Exchange: 0.0094 kilogram 'crushed stone, from quarry in Europe, excluding transport to the Netherlands' (kilogram, NL, None) to 'A1, pavement, materials, A' (kilogram, NL, None)>
Exchange: 0.0342 kilogram 'gravel production, crushed' (kilogram, RoW, None) to 'A1, pavement, materials, A' (kilogram, NL, None)>
Exchange: 0.0021 kilogram 'cellulose fibre production' (kilogram, RoW, None) to 'A1, pavement, materials, A' (kilogram, NL, None)>
Exchange: 0.027 kilogram 'medium filler' (kilogr

The exchanges include a "comments" field

In [11]:
for e in pavement_mats.technosphere():
    if "comments" in e.as_dict():
        # print(e["comments"])
        pprint(e["comments"])

'rap'
'bitumen'
'crushed stone'
'own material'
'crushed sand'
'drip resistant material'
'filler'


In [12]:
def exchange_coords(exchange):
    """create a tuple with the numerical ids of the input and output activites of an exchange."""
    input_activity = bd.get_activity(exchange["input"])
    output_activity = bd.get_activity(exchange["output"])
    return (input_activity.id, output_activity.id)

In [13]:
# We can create tuples of coords for every non "production" exchange of an activity like this.
for e in pavement_mats.technosphere():
    print(exchange_coords(e))

(25957, 25964)
(24816, 25964)
(25959, 25964)
(25959, 25964)
(20635, 25964)
(6027, 25964)
(25960, 25964)


### Parse all the pre-generated samples.

In [14]:
samples_df = pd.read_excel(
    "Samples.xlsx", sheet_name=pavement_complete_a["name"], dtype=float, index_col=0
)
samples_df.head()

,A1_bitumen,A1_crushedsand,A1_crushedstone3,A1_asphaltgranulate,A1_ownmaterial,A1_mediumfiller,A1_dripresistantmaterial,A2_bitumen,A2_crushedsand_t,A2_crushedsand_iv,...,A2_mediumfiller,A2_dripresistantmaterial,A3_naturalgas,A3_electricity,A3_diesel,A4_distance,A5_construction,C1_removal,C2_distance,C3_craneandshovel
0,0.042769,0.033898,0.607428,0.309227,0.008904,0.026087,0.002093,0.456430,0.039696,0.862846,...,0.097782,0.126065,0.007560,0.005661,0.000125,0.027180,0.000325,0.000807,0.071503,0.000199
1,0.039400,0.033167,0.587766,0.286452,0.008751,0.025986,0.002148,0.208577,0.057187,0.463359,...,0.198974,0.218366,0.007652,0.005420,0.000114,0.056457,0.000331,0.000753,0.022461,0.000184
2,0.040493,0.036037,0.558587,0.284832,0.009144,0.027256,0.002093,0.242726,0.043410,0.597687,...,0.154386,0.219272,0.007962,0.005683,0.000118,0.060631,0.000333,0.000827,0.040531,0.000179
3,0.039338,0.034951,0.585766,0.296381,0.009676,0.025594,0.002071,0.282429,0.018890,0.773004,...,0.175106,0.192699,0.008075,0.005632,0.000122,0.056707,0.000318,0.000830,0.050517,0.000187
4,0.040019,0.034629,0.582246,0.286346,0.009178,0.026117,0.002178,0.187741,0.041425,0.534964,...,0.087964,0.167227,0.008339,0.005551,0.000124,0.023251,0.000316,0.000746,0.058315,0.000184


In [15]:
samples_df.columns

Index(['A1_bitumen', 'A1_crushedsand', 'A1_crushedstone3',
       'A1_asphaltgranulate', 'A1_ownmaterial', 'A1_mediumfiller',
       'A1_dripresistantmaterial', 'A2_bitumen', 'A2_crushedsand_t',
       'A2_crushedsand_iv', 'A2_crushedstone3_t', 'A2_crushedstone3_iv',
       'A2_crushedstone3_sv', 'A2_ownmaterial_t', 'A2_ownmaterial_iv',
       'A2_mediumfiller', 'A2_dripresistantmaterial', 'A3_naturalgas',
       'A3_electricity', 'A3_diesel', 'A4_distance', 'A5_construction',
       'C1_removal', 'C2_distance', 'C3_craneandshovel'],
      dtype='object')

Each sample column is meant to be associated to one phase (A1, A2, etc.) and an inputs of a pavement exchange.
We can map the comment of the exchange to an existing samples column name roughly like this:

📌 I figured out, the above is not necessarily true. For example, the A4 phase, has 2 inputs: EURO5 & EURO6 trucks. 
    ❓ How is this supposed to be applied ? the `A4_distance` samples are to be applied to each truck ?


In [16]:
def sample_name_for_exchange(exchange, phase):
    """Return the sample name to use for this exchange."""
    all_names_mapping = {
        "A1": {
            "bitumen": "A1_bitumen",
            "crushed sand": "A1_crushedsand",
            "crushed stone": "A1_crushedstone3",
            "rap": "A1_asphaltgranulate",
            "own material": "A1_ownmaterial",
            "filler": "A1_mediumfiller",
            "drip resistant material": "A1_dripresistantmaterial",
        },
        "A2": {
            "bitumen": "A2_bitumen",
            "filler": "A2_mediumfiller",
            "drip resistant material": "A2_dripresistantmaterial",
        },
    }

    if "A1" == phase:
        names_mapping = all_names_mapping[phase]
        col_name = names_mapping[exchange["comments"]]
    if "A2" == phase:
        names_mapping = all_names_mapping[phase]
        if exchange["comments"] in names_mapping:
            return names_mapping[exchange["comments"]]
        else:
            input_activity = bd.get_activity(exchange["input"])
            comments = exchange["comments"]
            var_name = exchange["comments"].replace(" ", "")
            if var_name == "crushedstone":
                var_name = var_name + "3"
            ref_prod = input_activity["reference product"]
            if ref_prod == "transport, freight, sea, ferry":
                suffix = "sv"
            if ref_prod == "transport, freight, inland waterways, barge":
                suffix = "iv"
            if ref_prod == "transport, freight, lorry, unspecified":
                suffix = "t"
            col_name = f"{phase}_{var_name}_{suffix}"
    if "A3" == phase:
        input_activity = bd.get_activity(exchange["input"])
        ref_prod = input_activity["reference product"]
        if ref_prod == "electricity, low voltage":
            col_name = f"{phase}_electricity"
        elif ref_prod == "heat, district or industrial, natural gas":
            col_name = f"{phase}_naturalgas"
        elif ref_prod == "diesel, burned in building machine":
            col_name = f"{phase}_diesel"
    if "A4" == phase or "C2" == phase:
        input_activity = bd.get_activity(exchange["input"])
        ref_prod = input_activity["reference product"]
        if ref_prod in [
            "transport, freight, lorry >32 metric ton, EURO5",
            "transport, freight, lorry >32 metric ton, EURO6",
        ]:
            col_name = f"{phase}_distance"
    if "A5" == phase:
        col_name = f"{phase}_construction"
    if "C1" == phase:
        comments = exchange["comments"]
        var_name = exchange["comments"].lower()
        col_name = f"{phase}_{var_name}"
    if "C3" == phase:
        col_name = f"{phase}_craneandshovel"

    return col_name

In [17]:
def build_coords_sample(activity, samples_df):
    """Build the coords + sample dictionnary for all phases of activity.

    The activities are the top level "pavement complete" producing ones:

    [0]: DZOAB, A, PVI (pavement, complete, NL)
    [1]: DZOAB, B, PVI (pavement, complete, NL)
    [2]: DZOAB, B (pavement, complete, NL)
    [3]: DZOAB, A (pavement, complete, NL)

    """
    # Make sure we have a valid activity to start with
    assert "pavement, complete" == activity["reference product"]
    coords_samples_map = {}

    # the phases are for example: A1, pavement, materials, A
    # We must create the coords_sample for the inputs of each "phase"
    phase_exchanges = [input for input in activity.technosphere()]

    for phase_exchange in phase_exchanges:
        input_activity = bd.get_activity(phase_exchange["input"])
        # pprint(input_activity)
        # mapping = build_mapping(input_activity)
        # print(mapping)
        # Extract the "A1", "A2", etc. substring from the phase name
        current_phase = input_activity["name"].split(",")[0]
        for exchange in input_activity.technosphere():
            col_name = sample_name_for_exchange(exchange, current_phase)
            # print(f"{exchange}\n\t 👉 {col_name}")
            coords = exchange_coords(exchange)
            coords_samples_map[exchange_coords(exchange)] = samples_df[
                col_name
            ].values  # numpy.ndarray
    return coords_samples_map

In [18]:
# coords_samples_map = build_coords_sample(pavement_complete_a, samples_df)

In [20]:
# This is how to build the samples but only for the pavement materials activity
coords_samples_map = {}
for e in pavement_mats.technosphere():
    col_name = sample_name_for_exchange(e, "A1")
    print(f"{e} -> {col_name}")
    coords_samples_map[exchange_coords(e)] = samples_df[
        col_name
    ].values  # numpy.ndarray

Exchange: 0.3 kilogram 'asphalt granulate, free of burden' (kilogram, NL, None) to 'A1, pavement, materials, A' (kilogram, NL, None)> -> A1_asphaltgranulate
Exchange: 0.0412 kilogram 'bitumen adhesive compound production, hot' (kilogram, RER, None) to 'A1, pavement, materials, A' (kilogram, NL, None)> -> A1_bitumen
Exchange: 0.5861 kilogram 'crushed stone, from quarry in Europe, excluding transport to the Netherlands' (kilogram, NL, None) to 'A1, pavement, materials, A' (kilogram, NL, None)> -> A1_crushedstone3
Exchange: 0.0094 kilogram 'crushed stone, from quarry in Europe, excluding transport to the Netherlands' (kilogram, NL, None) to 'A1, pavement, materials, A' (kilogram, NL, None)> -> A1_ownmaterial
Exchange: 0.0342 kilogram 'gravel production, crushed' (kilogram, RoW, None) to 'A1, pavement, materials, A' (kilogram, NL, None)> -> A1_crushedsand
Exchange: 0.0021 kilogram 'cellulose fibre production' (kilogram, RoW, None) to 'A1, pavement, materials, A' (kilogram, NL, None)> -> A1

Here is a dictionnary with the coords + vector with the samples.

In [25]:
for k, v in coords_samples_map.items():
    print(f"{k} --array-> {v}")

(25957, 25964) --array-> [0.30922678 0.28645184 0.28483178 ... 0.30004993 0.28893112 0.29320903]
(24816, 25964) --array-> [0.04276906 0.03939971 0.04049269 ... 0.03997305 0.04136191 0.04123304]
(25959, 25964) --array-> [0.00890414 0.00875113 0.00914362 ... 0.00933997 0.00974056 0.0098617 ]
(20635, 25964) --array-> [0.03389766 0.03316733 0.03603696 ... 0.03230746 0.03355975 0.03485532]
(6027, 25964) --array-> [0.00209336 0.00214763 0.00209298 ... 0.00207337 0.0021744  0.00219949]
(25960, 25964) --array-> [0.02608693 0.02598636 0.02725618 ... 0.02811418 0.02555453 0.02638917]


In [27]:
# cml_methods = [m for m in bd.methods if m[0] == "CML v4.8 2016"]
cml_method_gwp = (
    "CML v4.8 2016",
    "climate change",
    "global warming potential (GWP100)",
)

## Regular LCA from activity + method

In [28]:
a_lca = bc.LCA({pavement_complete_a: 1}, cml_method_gwp)

In [29]:
a_lca.lci()
a_lca.lcia()
a_lca.score

0.16297396843731118

## LCA using datapackages

In [30]:
# the following function returns:
# indexed_demand, data_objs, remapping_dicts
# that follow the data package spec
indexed_demand, data_objs, remapping_dicts = bd.prepare_lca_inputs(
    {pavement_complete_a: 1}, cml_method_gwp
)

In [31]:
indexed_demand

{25976: 1}

In [32]:
data_objs

In [33]:
for i in data_objs:
    print(i.metadata["name"])

ecoinvent-3.9.1-cutoff
biosphere3
asphalt
cml-v48-2016cg.231d6e8f8b1c199a47182515eba4032e.zip


In [34]:
dp_lca = bc.LCA(demand=indexed_demand, data_objs=data_objs)
dp_lca.lci()
dp_lca.lcia()
dp_lca.score

0.16297396843731118

In [35]:
biosphere_dp = [dp for dp in data_objs if dp.metadata["name"] == "biosphere3"].pop()
biosphere_dp.metadata["name"]

'biosphere3'

In [36]:
ei_dp = [dp for dp in data_objs if dp.metadata["name"] == EI_DB_NAME].pop()
ei_dp.metadata["name"]

'ecoinvent-3.9.1-cutoff'

In [37]:
fg_dp = [dp for dp in data_objs if dp.metadata["name"] == "asphalt"].pop()
fg_dp.metadata["name"]

'asphalt'

In [38]:
pprint(fg_dp.metadata)

{'combinatorial': False,
 'created': '2024-02-20T08:01:29.701385Z',
 'id': '9df22ebd67ac4f8d90a3834f7825ca08',
 'licenses': [{'name': 'ODC-PDDL-1.0',
               'path': 'http://opendatacommons.org/licenses/pddl/',
               'title': 'Open Data Commons Public Domain Dedication and '
                        'License v1.0'}],
 'name': 'asphalt',
 'profile': 'data-package',
 'resources': [{'category': 'vector',
                'format': 'npy',
                'group': 'asphalt_inventory_geomapping_matrix',
                'kind': 'indices',
                'matrix': 'inv_geomapping_matrix',
                'mediatype': 'application/octet-stream',
                'name': 'asphalt_inventory_geomapping_matrix.indices',
                'nrows': 20,
                'path': 'asphalt_inventory_geomapping_matrix.indices.npy',
                'profile': 'data-resource'},
               {'category': 'vector',
                'format': 'npy',
                'group': 'asphalt_inventory_geoma

In [39]:
dp_correlated = bwp.create_datapackage(sequential=True)

In [40]:
samples_indices = [coord for coord in coords_samples_map.keys()]
pprint(samples_indices)

[(25957, 25964),
 (24816, 25964),
 (25959, 25964),
 (20635, 25964),
 (6027, 25964),
 (25960, 25964)]


In [41]:
# Prepare the data_array, it must be an array of arrays
samples_values = np.array([coords_samples_map[coord] for coord in samples_indices])
pprint(samples_values)

array([[0.30922678, 0.28645184, 0.28483178, ..., 0.30004993, 0.28893112,
        0.29320903],
       [0.04276906, 0.03939971, 0.04049269, ..., 0.03997305, 0.04136191,
        0.04123304],
       [0.00890414, 0.00875113, 0.00914362, ..., 0.00933997, 0.00974056,
        0.0098617 ],
       [0.03389766, 0.03316733, 0.03603696, ..., 0.03230746, 0.03355975,
        0.03485532],
       [0.00209336, 0.00214763, 0.00209298, ..., 0.00207337, 0.0021744 ,
        0.00219949],
       [0.02608693, 0.02598636, 0.02725618, ..., 0.02811418, 0.02555453,
        0.02638917]])


In [47]:
dp_correlated.add_persistent_array(
    matrix="technosphere_matrix",
    indices_array=np.array(samples_indices, dtype=bwp.INDICES_DTYPE),
    data_array=samples_values,
    # We flip the signs of the samples
    # to obey bw's convention
    flip_array=np.array([True for _ in range(len(samples_values))]),
)

### Build the new data objects to re-do the LCA

In [48]:
data_objs.append(dp_correlated)

In [49]:
dp_lca = bc.LCA(demand=indexed_demand, data_objs=data_objs, use_arrays=True)
dp_lca.lci()
dp_lca.lcia()
dp_lca.score

0.1536399277298662

In [53]:
scores = []
for _ in autonotebook.tqdm(range(400)):
    next(dp_lca)
    scores.append(dp_lca.score)

  0%|          | 0/400 [00:00<?, ?it/s]

In [54]:
scores_a = np.array(scores)
scores_a.mean()

0.15269161733122613

In [55]:
scores_a_s = pd.Series(scores_a)

In [56]:
scores_a_s.describe()

count    400.000000
mean       0.152692
std        0.001239
min        0.148782
25%        0.151874
50%        0.152704
75%        0.153548
max        0.156227
dtype: float64